In [ ]:
import sys
import time
import os
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from astropy.io import fits
import matplotlib.pyplot as plt

% matplotlib inline
%config InlineBackend.figure_format = 'retina'


class Watcher:
    DIRECTORY_TO_WATCH = "."
    NUM_TO_SKIP = 5
    

    def __init__(self,dirName, numToSkip):
        self.observer = Observer()
        self.DIRECTORY_TO_WATCH = dirName
        self.NUM_TO_SKIP=numToSkip
        print("Monitoring - %s render each %d."% (self.DIRECTORY_TO_WATCH , self.NUM_TO_SKIP))


    def run(self):
        event_handler = Handler(self.NUM_TO_SKIP)
        self.observer.schedule(event_handler, self.DIRECTORY_TO_WATCH, recursive=True)
        self.observer.start()
        try:
            while True:
                time.sleep(5)
        except:
            self.observer.stop()
            print("Error")

        self.observer.join()

class Handler(FileSystemEventHandler):
    COUNT=0
    NUM_TO_SKIP=0
    
    def __init__(self,skip):
        Handler.COUNT=0
        Handler.NUM_TO_SKIP=skip
    
    @staticmethod
    def on_any_event(event):
        if event.is_directory:
            return None

        elif event.event_type == 'created':
            # Count fits files and render some of them .. 
            print("Received - %s." % event.src_path)
            if (event.src_path.endswith(".fits") or event.src_path.endswith(".fits.gz") ):
                Handler.COUNT = Handler.COUNT + 1
                if (Handler.COUNT % Handler.NUM_TO_SKIP == 0):
                    Handler.render(event.src_path)
            else:
                print("Skipping - %s." % event.src_path)

    @staticmethod
    def render(filePath):
        print("Rendering - %s." % filePath)
        image_data = fits.getdata(filePath)
        plt.rcParams['figure.figsize'] = [10, 10]
        plt.imshow(image_data, cmap='gray')
        plt.colorbar()
        plt.show()
        
        

In [ ]:
    d = "/home/womullan/notebooks/tmp"
    skip = 2
    w = Watcher(d,skip)
    w.run()
